In [1]:
!pip install pandas -q
!pip install mysql-connector-python -q
!pip install streamlit -q

In [62]:
%%writefile exe_streamlit.py

from tkinter.tix import COLUMN
from PIL import Image

import altair as alt
import numpy as np
import pandas as pd
import mysql.connector
import streamlit as st
import re


class Car:
    def __init__(self, brand, model_name, detail, price, image):
        self.brand = brand
        self.model_name = model_name
        self.detail = detail
        self.price = price
        self.image = image
    @staticmethod
    def from_sql(car_data):
        print(car_data)
        return Car(*car_data)

db = mysql.connector.connect(
    host="localhost",
    user="root",      
    password="root1234",   
    database="car_project"  # 데이터베이스 변경
)

cursor = db.cursor()

#-------------------------------------------------------------------
#세부 정보 셀렉트박스용 데이터
all_model = [' ']
try:
    cursor = db.cursor()
    query = f'''
    select
    	join_model.model_name as model
    from
    	(SELECT m.model_id, m.model_name, ms.sales_model_id, ms.month, ms.sales_month
    	 FROM model m
         JOIN model_sales ms ON m.model_id = ms.sales_model_id) as join_model;
    '''
    cursor.execute(query)
    for model in cursor.fetchall():
        model_name = model[0]  # 튜플에서 첫 번째 요소만 가져옴
        if model_name not in all_model:
            all_model.append(model_name)  # 쉼표 없이 문자열만 추가
except Exception as e:
    print(e)
#-------------------------------------------------------------------
# 모델별 데이터 수집

data_model = []
data_sales = []
data_sales_month = []

def n_data(db,input_month):
    try:
        if input_month == 13:
            cursor = db.cursor()
            query = f'''
            select
            	join_model.model_name as model,
                sum(join_model.sales_month) as sum_sales
            from
            	(SELECT m.model_id, m.model_name, ms.sales_model_id, ms.month, ms.sales_month
            	 FROM model m
                 JOIN model_sales ms ON m.model_id = ms.sales_model_id) as join_model
            where (join_model.month <> 0)
            group by model
            order by sum_sales desc limit 10;
                '''
            cursor.execute(query)
                
            for m_name,sales in cursor.fetchall():
                data_model.append(m_name)
                data_sales.append(sales)
        else:
            cursor = db.cursor()
            query = f'''
            select
                join_model.model_name as model,
                sum(join_model.sales_month) as sum_sales,
                join_model.month as month
            from
                (SELECT m.model_id, m.model_name, ms.sales_model_id, ms.month, ms.sales_month
            	 FROM model m
                 JOIN model_sales ms ON m.model_id = ms.sales_model_id) as join_model
            where join_model.month = {input_month}
            group by model
            order by sum_sales desc limit 10;
                '''
            cursor.execute(query)
                
            for m_name,sales,sales_month in cursor.fetchall():
                data_model.append(m_name)
                data_sales.append(sales)
                data_sales_month.append(sales_month)
        
    except Exception as e:
        print(e)
# # n_data() end
#-------------------------------------------------------------------
# 브랜드별 데이터 수집

brand_name = []
brand_sum_sales = []

def b_data(db):
    try:
        cursor = db.cursor()
        query = f'''
        SELECT
        	brand_name,
            sales_year
        FROM brand
        JOIN brand_sales ON brand.brand_id = brand_sales.sales_brand_id;
        '''
        cursor.execute(query)
        
        for brand, sum_sales in cursor.fetchall():
            brand_name.append(brand)
            brand_sum_sales.append(sum_sales)
    except Exception as e:
        print(e)
    
# b_Data end
#-------------------------------------------------------------------
# FAQ 데이터
question_1 = []
answer_1 = []
question_2 = []
answer_2 = []

try:
    cursor = db.cursor()
    query = '''
    select * from faq;
    '''

    cursor.execute(query)

    for brand_id, question, answer in cursor.fetchall():
        if brand_id == 1:
            question_1.append(question)
            answer_1.append(answer)
        if brand_id == 2:
            question_2.append(question)
            answer_2.append(answer)
except Exception as e:
    print(e)
#-------------------------------------------------------------------

# 메인 구동

option = st.sidebar.selectbox(
    ' ',
    ('메인','모델별 최근 판매 순위','브랜드별','세부정보','FAQ')
)
if option == '메인':
    st.title('AZ A:small[:gray[utovi]]Z')
    st.subheader('2024년 자동차 판매 데이터 정리', divider = 'gray')
    image = 'https://cdn.discordapp.com/attachments/1350036369979736155/1351422897020993536/file-T1xvczouyE18e67P9xjs93.png?ex=67da5226&is=67d900a6&hm=a4d53a173ca8d7bafd2db058b1ef062525c0366fd2a33ff0fb71d39fe2d9a0bf&'
    st.image(image, width=600)

#-------------------------------------------------------------------
# 좌측 사이드바 메뉴

# 최근 1년 판매순위(모델별)
if option == '모델별 최근 판매 순위':
    st.title("모델별 최근 판매 순위")
    st.subheader('판매량 Top 10')
    # 줄바꿈으로 차트 위치 조정
    st.text("\n")
    st.text("\n")
    st.text("\n")
    st.text("\n")
    m_month_data = st.selectbox(
        '원하는 달(기간)을 정해주세요',
        ('2024년','1월','2월','3월','4월','5월','6월','7월','8월','9월','10월','11월','12월')
    )
    st.text("\n")
    st.text("\n")
    if m_month_data == '2024년':
        n_data(db,13)
        
        data = pd.DataFrame(
            {
                '모델명': data_model,  # x축 값
                '판매량': data_sales  # y축 값
            }
        )
    
        chart = alt.Chart(data).mark_bar().encode(
            x=alt.X('모델명:N', sort='-y', title='모델명'),  # N: Nominal (범주형 데이터)
            y=alt.Y('판매량:Q', title=f'2024년 전체 판매량', scale=alt.Scale(zero=True)),  # Q: Quantitative (수치형 데이터)
        ).properties(
            width=1000,  # 그래프 너비 설정
            height=400  # 그래프 높이 설정
        )
        st.altair_chart(chart, use_container_width=True)
    
    else:
        m_month_data = int(''.join(re.findall(r'\d+',m_month_data)))
        n_data(db, m_month_data)
    
        data = pd.DataFrame(
            {
                '모델명': data_model,  # x축 값
                '판매량': data_sales  # y축 값
            }
        )
    
        chart = alt.Chart(data).mark_bar().encode(
            x=alt.X('모델명:N', sort='-y', title='모델명'),  # N: Nominal (범주형 데이터)
            y=alt.Y('판매량:Q', title=f'{data_sales_month[0]}월 판매량', scale=alt.Scale(zero=True)),  # Q: Quantitative (수치형 데이터)
        ).properties(
            width=1000,  # 그래프 너비 설정
            height=400  # 그래프 높이 설정
        )
        st.altair_chart(chart, use_container_width=True)
        
#-------------------------------------------------------------------
# 최근 1년 판매순위(브랜드별 페이지)
if option == '브랜드별':
    st.title("브랜드별 페이지")
    st.subheader('2024 브랜드별 판매량 Top 10')
    st.text("\n")
    st.text("\n")
    st.text("\n")
    st.text("\n")
    st.text("\n")
    b_data(db)
    data = pd.DataFrame(
        {
            '브랜드': brand_name,
            '판매량': brand_sum_sales
        }
    )
    
    chart = alt.Chart(data).mark_bar().encode(
        x=alt.X('브랜드:N', sort='-y', title='브랜드'),  # N: Nominal (범주형 데이터)
        y=alt.Y('판매량:Q', title=f'2024년도 판매량', scale=alt.Scale(zero=True)),  # Q: Quantitative (수치형 데이터)
    ).properties(
        width=1000,  # 그래프 너비 설정
        height=400  # 그래프 높이 설정
    )
    st.altair_chart(chart, use_container_width=True)
    
#-------------------------------------------------------------------
# 세부정보 페이지
try:
    query = '''
    select
    b.brand_name,
    m.model_name,
    m.detail,
    m.price,
    m.image
    
    from model as m
    join brand as b
    on m.model_brand_id = b.brand_id
    ;
    '''
    cursor.execute(query)
    temp = cursor.fetchall()
    cars_list = [Car.from_sql(car) for car in temp]
    print(cars_list)
except Exception as e:
    print(e)

if option == '세부정보':
    st.title("모델 별 세부 정보")
    
    cols = st.columns(3)

    card_style = """
        <style>
            .card {
                border: 2px solid #000; /* 테두리 */
                border-radius: 15px; /* 둥근 모서리 */
                padding: 16px;
                text-align: center;
                box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1); /* 그림자 효과 */
                margin-bottom: 5px;
                min-height: 500px; /* 최소 높이 설정 */
                display: flex;
                flex-direction: column;
                justify-content: space-between; /* 일정한 간격 유지 */
            }
            .car-detail {
                color: #9E9D9D; /* 연한 회색 (사용자 지정 색상) */
                font-size: 8px; /* 작은 글씨 */
            }
            .model-name {
                font-size: 12px; /* 모델 이름의 글씨 크기 */
                font-weight: bold; /* 모델 이름을 굵게 */
            }
            .car-price {
                font-size: 8px; /* 가격 글씨 크기 */
                color: #333; /* 가격 색상 */
            }
            .car-image {
                max-width: 100%; /* 이미지 크기 조정 */
                height: auto; /* 비율에 맞게 자동 조정 */
                border-radius: 10px; /* 이미지에 둥근 모서리 */
            }
        </style>
    """
    st.markdown(card_style, unsafe_allow_html=True)
    
    for i, car in enumerate(cars_list):
        with cols[i % 3]:  # 3개씩 정렬
            st.markdown(
                f"""
                <div class="card">
                    <img src="{car.image}" class="car-image" alt="Car Image"> <!-- 이미지 추가 -->
                    <h3 class="model-name">{car.model_name}</h3> <!-- 모델 이름 스타일 -->
                    <p class="car-detail">{car.detail}</p>  <!-- 지정된 색상 적용 -->
                    <h4 class="car-price">{car.price:,}원</h4> <!-- 가격 스타일 -->
                </div>
                """,
                unsafe_allow_html=True
            )

#-------------------------------------------------------------------
# FAQ 페이지


if option == 'FAQ':

    st.title("FAQ")
    col1, col2 = st.columns([1, 2])
    with col2:
        brand_input = st.selectbox(
        '',
        ('현대','기아')
    )

    #현대 FAQ
    if brand_input == '현대':
        with col1:
            st.markdown("\n\n")
            st.image('https://autoimg.danawa.com/photo/brand/303_90.png', width=100)
        for i in range(len(question_1)):
            with st.expander(f'{question_1[i]}'):
               st.write(f'{answer_1[i]}')
    # 기아 FAQ
    if brand_input == '기아':
        with col1:
            st.markdown("\n\n")
            st.image('https://autoimg.danawa.com/photo/brand/307_90.png', width=100)
        for i in range(len(question_2)):
            with st.expander(f'{question_2[i]}'):
               st.write(f'{answer_2[i]}')

Overwriting exe_streamlit.py


In [ ]:
!streamlit run exe_streamlit.py